In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
test=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
sub=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv')

In [ ]:
alldata=pd.concat([train,test])
alldata

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
TK=Tokenizer()
TK.fit_on_texts(alldata['target'])
TK.word_index

In [ ]:
target=TK.texts_to_sequences(alldata['target'])
target

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_target=pad_sequences(text,maxlen=4)
pad_target

In [ ]:
train2=pad_target[:len(train)]
test2=pad_target[len(train):]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train2,train['score'],test_size=0.2,random_state=42,stratify=train['score'])

In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *

In [ ]:
alldata['score'].nunique()

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True, random_state=26)
result=0
for train_index, valid_index in kf.split(train2):
    x_train = train2[train_index]
    x_valid = train2[valid_index]
    y_train = train['score'].iloc[train_index]
    y_valid = train['score'].iloc[valid_index]
    
    model=Sequential()
    model.add(Embedding(len(TK.word_index)+1,input_length=30,output_dim=5))
    model.add(LSTM(32))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='acc')
    model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_valid,y_valid))
    result+=model.predict(test2)/5

In [ ]:
sub['score']=result
sub.to_csv('submission.csv',index=False)